In [3]:
# Import library
import requests
import re
import os
from urllib.request import urlretrieve

In [9]:
# Get url info
def get_url(key):
    
    url = 'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=' + key + '&btnG='
    return url

geturl('python')

'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=python&btnG='

In [18]:
# set header in case that the service identify that you are a code
headers = {
    'user-agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
    'Referer': 'https://googleads.g.doubleclick.net/' 
}

In [19]:
# Get GOI lists
def get_paper_link(headers, key):
    response = requests.get(url=get_url(key), headers=headers)
    res1_data = response.text
    #Find paper link
    paper_link = re.findall(r'<h3 class=\"t c_font\">\n +\n +<a href=\"(.*)\"',
                            res1_data) ## RE(Regular expression)
    
    doi_list = []
    for link in paper_link:
        paper_link = 'http:' + link
        response2 = requests.get(url=paper_link, headers=headers)
        res2_data = response2.text
        # Get DOI
        try:
            paper_doi = re.findall(r'\'doi\'}\">\n +(.*?)\n ', res2_data)
            if str(10) in paper_doi[0]:
                doi_list.append(paper_doi)
                
        except:
            pass
    return doi_list


get_paper_link(headers,'python')

[]

In [15]:
# download from sci-hub
def doi_download(headers, key):
    doi_list = get_paper_link(headers,key)
    #print(doi_list)
    for doi in doi_list:
        doi_link = 'hetts://sci-hub.tf/' + doi[0]
        print(doi_link)
        
        if 'https:' not in doi_link:
            doi_link = 'https:' + doi_link
            
        res = requests.get(url=doi_link, headers=headers)
        
        try:
            down_link = re.findall(r'<iframe.*?src="(.*?)" id=.*?<\/iframe>',
                               res.text)[0]
            
            print(down_link)
            r = requests.get(url=down_link, headers=headers)
            
            path = doi_link.split('/')[-1] + '.pdf'
            with open(path, 'wb') as f:
                f.write(r.content)
                
                print('Done!' + doi_link.split('/')[-1])
        except:
            print("The paper is empty!")
            pass

In [17]:
#Search and download
key = input("Please input the key words:")
doi_download(headers, key)

Please input the key words:python
